In [51]:
import tensorflow as tf # v2 
import numpy as np
import random


In [52]:
"""
データをつくる
X(input) -> t(teach)
"""
def create_data(nb_of_samples, sequence_len):
    X = np.zeros((nb_of_samples, sequence_len))
    for row_idx in range(nb_of_samples):
        X[row_idx,:] = np.around(np.random.rand(sequence_len)).astype(int)
    # Create the targets for each sequence
    t = np.sum(X, axis=1)
    return X, t

def make_prediction(nb_of_samples,sequence_len):
    xs, ts = create_data(nb_of_samples, sequence_len)
    return np.array([[[y] for y in x] for x in xs]), np.array([[x] for x in ts])

- tf.truncated_normal:±2σの切断正規分布からランダムに取り出したテンソルを生成する
    - shape: 戻り値のテンソルの次元 (1次元のintegerリストもしくはテンソルを指定）
    - mean: 生成する切断正規分布の平均値
    - stddev: 生成する切断正規分布の標準偏差
    - dtype: 戻り値のテンソルの型
    - seed: integerを指定すると、sessionが変わっても都度同じ値を抽出するようになる
    - name: operationの名前
- inference
    - 仮に[a-z]{4}の文字列から次の[a-z]{1}を予測する場合、
        - chunk_size:4+1->5
        - data_size:26 -> このままinput_layer_sizeになる
- tf.reshape
    - もとの形をR*C = Nとして、[-1,2]ならば−1の部分はN/2が自動代入
- tf.transpose
    - 転置。順序を指定して変更
- tf.matmul
    - (w, x)はwとxの行列積
- tf.split(V,n,axis)
    - Vをnで分割.axisで指定した次元で分割

In [63]:
"""
実際のネットワークの構成を記述する部分
"""
def inference(input_ph, istate_ph):
    """
    input_data: (batch_size, length_of_sequences, num_of_input_nodes = vocabulary_size) 次元のテンソル
    initial_state: (batch_size, hidden_layer_size) 次元の行列
    """
    with tf.name_scope("inference") as scope:

        # 重みとバイアスの初期化。
        weight1_var = tf.Variable(tf.random.truncated_normal([num_of_input_nodes, num_of_hidden_nodes], stddev=0.1), name="weight1")
        weight2_var = tf.Variable(tf.random.truncated_normal([num_of_hidden_nodes, num_of_output_nodes], stddev=0.1), name="weight2")
        bias1_var = tf.Variable(tf.random.truncated_normal([num_of_hidden_nodes], stddev=0.1), name="bias1")
        bias2_var = tf.Variable(tf.random.truncated_normal([num_of_output_nodes], stddev=0.1), name="bias2")

        #### LSTMのcellに入る前の最初の部分 ####
        # (length_of_sequences, batch_size, num_of_input_nodes=vocabulary_size)へ変換
        in1 = tf.transpose(input_ph, [1, 0, 2]) 
        # (length_of_sequences * batch_size), num_of_input_nodes)に変換。これで入力可能な状態になる
        in2 = tf.reshape(in1, [-1, num_of_input_nodes]) 
        # 重みWとバイアスBを適用。 
        #(length_of_sequences* batch_size)* num_of_input_nodes)が
        #(length_of_sequences* batch_size)* num_of_hidden_nodes)へ
        # LSTMユニット内でのセル更新式．に相当
        in3 = tf.matmul(in2, weight1_var) + bias1_var
        #リストに分割
        # (length_of_sequences * batch_size) *  num_of_hidden_nodes
        # batch_size * length_of_sequences * num_of_hidden_nodes
        in4 = tf.split(in3, length_of_sequences, 0) 

        #### LSTMのcell ####
        # LSTMCell：中間層にnum_of_hidden_nodeだけニューロンを生成
        # ｔｆにおけるcellは<状態と入力>で<新状態>を出力
        # LSTMにおける選択的な取り込みと選択的な出力はここで解決　
        cell = tf.compat.v1.nn.rnn_cell.LSTMCell(num_of_hidden_nodes, forget_bias=forget_bias, state_is_tuple=False)
        # 指定Cellと指定istate_phを初期状態とする層を自動生成。そこにinputを入れて自動で
        # 必要な数length_of_sequencesだけ生成
        rnn_output, states_op = tf.compat.v1.nn.static_rnn(cell, in4, initial_state=istate_ph)
        
        #### LSTMのcellから出た後の部分 ####
        # 最後に隠れ層から出力層につながる重みとバイアスを処理して終了です。
        # 出力層はlength_of_sequences個のベクトルを出力
        # outputs[-1] で最後の1文字だけを処理
        output_op = tf.matmul(rnn_output[-1], weight2_var) + bias2_var
        
        # Add summary ops to collect data
        w1_hist = tf.summary.histogram("weights1", weight1_var)
        w2_hist = tf.summary.histogram("weights2", weight2_var)
        b1_hist = tf.summary.histogram("biases1", bias1_var)
        b2_hist = tf.summary.histogram("biases2", bias2_var)
        output_hist = tf.summary.histogram("output",  output_op)
        results = [weight1_var, weight2_var, bias1_var,  bias2_var]
        
        return output_op, states_op, results


"""    
コストの計算
より詳細な損失関数の下記かたは
https://qiita.com/YudaiSadakuni/items/918b08e9c1fb497c96c3
"""
def loss(output_op, supervisor_ph):
    with tf.name_scope("loss") as scope:
        # tf.reduce_mean : 与えたリストに入っている数値の平均値
        # 下記はsquare_errorを算出
        square_error = tf.reduce_mean(tf.square(output_op - supervisor_ph))
        loss_op = square_error
        # lossをスカラーとして記録
        tf.compat.v1.summary.scalar("loss", loss_op)
        return loss_op
    
"""
新データを作成して、現時点での重みで精度を評価
"""
def calc_accuracy(output_op, prints=False):
    #　別途でデータを新規作成
    inputs, ts = make_prediction(num_of_prediction_epochs,length_of_sequences)
    pred_dict = {
        input_ph:  inputs,
        supervisor_ph: ts,
        istate_ph:    np.zeros((num_of_prediction_epochs, num_of_hidden_nodes * 2)),
    }
    #もう一度Sessoinを実行する。そしてoutput_opを実行し、値を返却
    output = sess.run([output_op], feed_dict=pred_dict)

    def print_result(i, p, q):
#         [print(list(x)[0]) for x in i]
        print("output: %f, correct: %d" % (p, q))

    if prints:
        [print_result(i, p, q) for i, p, q in zip(inputs, output[0], ts)]

    opt = abs(output - ts)[0]
    #square error の sum 
    total = sum([1 if x[0] < 0.05 else 0 for x in opt])
    #MSE
    print("accuracy %f" % (total / float(len(ts))))
    
    return output

"""
コスト関数の最適化関数の定義(optimizer)
optimizerに対して、最適化の対象を設定
"""
def training(loss_op):
    with tf.name_scope("training") as scope:
        training_op = optimizer.minimize(loss_op)
        return training_op


"""
各バッチで使用するデータを定義
"""
def get_batch(batch_size, X, t):
    rnum = [random.randint(0, len(X) - 1) for x in range(batch_size)]
    xs = np.array([[[y] for y in list(X[r])] for r in rnum])
    ts = np.array([[t[r]] for r in rnum])
    return xs, ts

In [64]:
"""
データの準備
"""
# 特徴量の次元
num_of_input_nodes = 1
#　隠れ層の次元
num_of_hidden_nodes = 80
#求めたい量の次元数
num_of_output_nodes = 1
## 時系列上のタイムステップ数
length_of_sequences = 10
#(補足)何回同一でーたを食わせるか
num_of_training_epochs = 5000
#バッチサイズ
size_of_mini_batch = 100

num_of_prediction_epochs = 100
learning_rate = 0.01
forget_bias = 0.8
num_of_sample = 1000

random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)


#最適化関数の定義
#今回は確率勾配
optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=learning_rate)

#こんかいは1次元でlength_of_sequencesの数列を作る。それをnum_of_sampleだけ作成する
X, t = create_data(num_of_sample, length_of_sequences)

In [65]:
print(X[0],t[0],'its mean sum of X[0] = t[0]')
print(len(t))
X.shape

[1. 1. 1. 1. 0. 1. 0. 1. 1. 0.] 7.0 its mean sum of X[0] = t[0]
1000


(1000, 10)

- tf.placeholder
    - プレースホルダーはデータが格納される入れ物。データは未定のままグラフを構築し、具体的な値は実行する時に与える。
    - 型, size
- tf.run
    - Feed_dict
        - placeholderとTensorに対応
        - https://ja.stackoverflow.com/questions/37259/tensorflow%E3%81%AEfeed-dict-x-data-x-val%E3%81%AE%E6%84%8F%E5%91%B3%E3%81%AB%E3%81%A4%E3%81%84%E3%81%A6
 

In [66]:


"""
tensorborsdのためにくくる
"""
with tf.Graph().as_default():
    ###  変数の定義  ###
    # input(特徴量の次元＊特徴量のタイムステップ数)
    input_ph = tf.compat.v1.placeholder(tf.float32, [None, length_of_sequences, num_of_input_nodes], name="input")
    # actual(求めたい量の次元数)
    supervisor_ph = tf.compat.v1.placeholder(tf.float32, [None, num_of_output_nodes], name="supervisor")
    # hidden
    istate_ph = tf.compat.v1.placeholder(tf.float32, [None, num_of_hidden_nodes * 2], name="istate")
    
    ### グラフを構築(呼び出し時には実際に計算される部分)  ###
    ### いずれも変数
    output_op, states_op, datas_op = inference(input_ph, istate_ph)
    ### 損失関数を定義（独自）  ###
    ### 正確には損失を計算するoperatorを定義。runしないと動かない
    loss_op = loss(output_op, supervisor_ph)
     ### optimizerを定義  ###
    ### 正確には損失を計算するoperatorを定義。runしないと動かない
    training_op = training(loss_op)

    # TensorBoard描画用の変数(Summry)
    # TensorBoardが利用するSummaryデータをmerge（合体）する
    summary_op = tf.compat.v1.summary.merge_all()
    
    # 定義した変数をここで初期化
    init =tf.compat.v1.initialize_all_variables()

    ## グラフの事項単位をセッション（Session）###
    with tf.compat.v1.Session() as sess:
        #Tensorflowの学習パラーメータのsave
        saver = tf.compat.v1.train.Saver()
        # TensorBoardが利用するSummaryデータのライターを作成。
        # 構築されたグラフを取得
        summary_writer =tf.compat.v1.summary.FileWriter("./tmp/tensorflow_log", graph=sess.graph)
        #Sessionオブジェクトを作成し、runメソッドを呼び出して対象を評価
        sess.run(init)


        for epoch in range(num_of_training_epochs):
            inputs, supervisors = get_batch(size_of_mini_batch, X, t)
            # istate_ph
            #  状態を伝達するのみ(length_of_sequences分の確保は不要)
            #   size_of_mini_batch *  num_of_hidden_node の順番もinput に揃える
            train_dict = {
                input_ph:      inputs,
                supervisor_ph: supervisors,
                istate_ph:     np.zeros((size_of_mini_batch, num_of_hidden_nodes * 2)),
            }
            ## グラフ（グラフと損失関数について）とデータを装填し、実行
            sess.run(training_op, feed_dict=train_dict)

            if (epoch) % 100 == 0:
                summary_str, train_loss = sess.run([summary_op, loss_op], feed_dict=train_dict)
                print("train#%d, train loss: %e" % (epoch, train_loss))
                
                #SummaryWriterにSummaryの評価結果を渡す
                summary_writer.add_summary(summary_str, epoch)
                
                #精度評価
                if (epoch) % 500 == 0:
                    calc_accuracy(output_op)

        calc_accuracy(output_op, prints=True)
        datas = sess.run(datas_op)
        saver.save(sess, "./data/model.ckpt")

train#0, train loss: 2.575568e+01
accuracy 0.000000
train#100, train loss: 8.676588e-01
train#200, train loss: 2.452996e-01
train#300, train loss: 7.731947e-02
train#400, train loss: 1.253954e-01
train#500, train loss: 1.023796e-01
accuracy 0.090000
train#600, train loss: 1.665974e-02
train#700, train loss: 4.783490e-02
train#800, train loss: 7.561896e-02
train#900, train loss: 9.487455e-02
train#1000, train loss: 5.012555e-02
accuracy 0.010000
train#1100, train loss: 2.318375e-02
train#1200, train loss: 9.734282e-03
train#1300, train loss: 1.749895e-02
train#1400, train loss: 3.587383e-02
train#1500, train loss: 7.807648e-03
accuracy 0.400000
train#1600, train loss: 6.797987e-02
train#1700, train loss: 1.606712e-02
train#1800, train loss: 1.291443e-02
train#1900, train loss: 5.111635e-02
train#2000, train loss: 1.061938e-02
accuracy 0.250000
train#2100, train loss: 1.278130e-02
train#2200, train loss: 1.422039e-02
train#2300, train loss: 1.759337e-02
train#2400, train loss: 3.107684e-